# Initialization

In [7]:
%load_ext autoreload
%autoreload 2

from jlib.transformer_char_predictor import TransformerCharPredictor
import jlib.data_utils as data_utils
import torch
import torch.nn as nn
import numpy as np
from torchprofile import profile_macs
text = data_utils.get_text('data/shakespeare.txt', redownload=False)
print(text[:50])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
First Citizen:
Before we proceed any further, hear


# SeqLen 10

In [8]:
seqlen = 10
def train_and_plot(seqlen: int):
    data = data_utils.gen_datasets(text, seqlen)
    train_data = data['train_dataset']
    val_data = data['val_dataset']
    alphabet: data_utils.Alphabet = data['alphabet']

    train_fetcher = data_utils.gen_data_loader(
        train_data,
        batch_size=len(train_data)//2048,
        workers = 6,
        cpu_prefetch= 10,
        gpu_prefetch=10
    )

    val_fetcher = data_utils.gen_data_loader(
        val_data,
        batch_size=len(val_data)//1024,
        workers = 6,
        cpu_prefetch= 10,
        gpu_prefetch=10
    )

    # model

    model = TransformerCharPredictor(
        alphabet_size = len(alphabet),
        max_len = seqlen,
        hidden_dim = 128,
        inner_dim = 1024,
        num_attn_heads = 2,
        num_attn_layers=2,
        cls_head_dims=[],
        dropout = 0.1
    )

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Model parameter count: {param_count:,}")
    
    

    # test_input = next(iter(train_fetcher))[0]

    # print(f"Model MACs: {profile_macs(model, test_input):,}")
    
# Model parameter count: 1,790,380
# Model MACs: 568,279,052



    model.train_model(
        epochs=100,
        train_fetcher=train_fetcher,
        val_fetcher=val_fetcher,
        optimizer = torch.optim.Adam,
        optimizer_kwargs={
            'lr': 3e-3,
            'betas': (0.9, 0.98),
            'eps': 1e-9,
            'weight_decay': 1e-5
        },
        min_accuracy=1,
        max_negative_diff_count=10,
        save_path=f'models/p2-{seqlen}.pth',
        stop_on_plateau=True,
    )

    fig = model.plot_training(f'Shakespeare Corpus, Sequence Length {seqlen}')
    fig.savefig(f'latex/images/p2-{seqlen}.png')
    
    del train_fetcher, val_fetcher, train_data, val_data, data, model, alphabet
    





In [9]:
train_and_plot(10)



Begin init data loader
Batch Size: 0.0331878662109375 MiB
Data Loader init time: 0.528920 s
Begin init fetcher
Fetcher init time: 0.741648 s
Begin init data loader
Batch Size: 0.0165557861328125 MiB
Data Loader init time: 0.343200 s
Begin init fetcher
Fetcher init time: 0.586229 s
Model parameter count: 676,417
Training TransformerCharPredictor

----------------------------------------------------------------------------------------------------------------------------------------------------
Begin Training
|       Epoch        |   Epoch Time (s)   |   Training Loss    |  Validation Loss   |Validation Accuracy |   Δ Accuracy (%)   |    Memory Usage    |
----------------------------------------------------------------------------------------------------------------------------------------------------
|         0          |     33.438522      |      0.679281      |      0.192975      |     94.564658      |      0.000000      |      0.591203      |
-----------------------------------------

KeyboardInterrupt: 

In [ ]:
train_and_plot(20)


In [ ]:
train_and_plot(30)